In [1]:
import json 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from collections import Counter


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_FEMALE_1610_1.json') as f:
    data = json.load(f)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_FEMALE_1610_1.json', 'r') as file:
    female_data_model1 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_MALE_1610_1.json', 'r') as file:
    male_data_model1 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_NEUT_1610_1.json', 'r') as file:
    neutral_data_model1 = json.load(file)

with open('//Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_FEMALE_1610_1.json', 'r') as file:
    female_data_model2 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_MALE_1610_1.json', 'r') as file:
    male_data_model2 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_NEUT_1610_1.json', 'r') as file:
    neutral_data_model2 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model1 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model2 = json.load(file)

In [3]:
def aggregate_data(data, gender, model):
    aggregated_data = []

    for prompt, adjectives in data.items():
        for adjective, details in adjectives.items():
            aggregated_data.append({
                "Prompt": prompt,
                "Descriptor": adjective,
                "Logit": details["logit"],
                "Antonym": details["antonym"],
                "Antonym Logit": details["antonym_logit"],
                "Favored": details["favored"],
                "Gender": gender,
                "Model": model  # Add model label
            })

    return aggregated_data

female_aggregated_model1 = aggregate_data(female_data_model1, "Female", "LLaMA-3-8b")
male_aggregated_model1 = aggregate_data(male_data_model1, "Male", "LLaMA-3-8b")
neutral_aggregated_model1 = aggregate_data(neutral_data_model1, "Neutral", "LLaMA-3-8b")

female_aggregated_model2 = aggregate_data(female_data_model2, "Female", "LLaMA-2-7b")
male_aggregated_model2 = aggregate_data(male_data_model2, "Male", "LLaMA-2-7b")
neutral_aggregated_model2 = aggregate_data(neutral_data_model2, "Neutral", "LLaMA-2-7b")


all_data = (female_aggregated_model1 + male_aggregated_model1 + neutral_aggregated_model1 +
            female_aggregated_model2 + male_aggregated_model2 + neutral_aggregated_model2)

df = pd.DataFrame(all_data)

diagnoses = [
    "schizophrenia", 
    "depression", 
    "generalised anxiety disorder", 
    "bipolar disorder", 
    "borderline personality disorder", 
    "a mental illness", 
    "antisocial personality disorder", 
    "post-traumatic stress disorder", 
    "obsessive-compulsive disorder", 
    "dissociative identity disorder"
]

def extract_diagnosis(prompt):
    for diagnosis in diagnoses:
        if diagnosis in prompt.lower():  
            return diagnosis
    return None  

reshaped_data = []

for _, row in df.iterrows():
    diagnosis = extract_diagnosis(row['Prompt'])  
    
    reshaped_data.append({
        'Prompt': row['Prompt'],  
        'adjective': row['Descriptor'],
        'logit': row['Logit'],
        'label': 'negative',  
        'Gender': row['Gender'],
        'Model': row['Model'],
        'diagnosis': diagnosis  
    })
    
    reshaped_data.append({
        'Prompt': row['Prompt'],  
        'adjective': row['Antonym'],
        'logit': row['Antonym Logit'],
        'label': 'positive',  
        'Gender': row['Gender'],
        'Model': row['Model'],
        'diagnosis': diagnosis  
    })

reshaped_df = pd.DataFrame(reshaped_data)

severity_mapping = {
    "schizophrenia": "severe",
    "depression": "not severe",
    "generalised anxiety disorder": "not severe",
    "bipolar disorder": "severe",
    "borderline personality disorder": "severe",
    "a mental illness": "not severe",  
    "antisocial personality disorder": "severe",
    "post-traumatic stress disorder": "not severe",
    "obsessive-compulsive disorder": "not severe",
    "dissociative identity disorder": "severe"
}

def add_severity_label(row):
    diagnosis = row['diagnosis']
    return severity_mapping.get(diagnosis, "unknown")  

reshaped_df['severity'] = reshaped_df.apply(add_severity_label, axis=1)

aggregated_df = reshaped_df.groupby(['Model', 'diagnosis', 'label', 'adjective'])['logit'].mean().reset_index()

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model1 = json.load(file)


with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model2 = json.load(file)

def aggregate_data_no_diagnosis(data, model):
    reshaped_data = []

    def extract_gender_from_pronouns(prompt):
        pronouns_to_gender = {
            "he": "Male",
            "him": "Male",
            "his": "Male",
            "she": "Female",
            "her": "Female",
            "hers": "Female",
            "they": "Neutral",
            "them": "Neutral",
            "theirs": "Neutral"
        }
        tokens = prompt.lower().split() 
        for pronoun, gender in pronouns_to_gender.items():
            if pronoun in tokens:  
                return gender
        return "Unknown"  

    for prompt, adjectives in data.items():
        gender = extract_gender_from_pronouns(prompt)  

        for adjective, details in adjectives.items():
            reshaped_data.append({
                'Prompt': prompt,
                'adjective': adjective,
                'logit': details["logit"],
                'label': 'negative', 
                'Gender': gender,
                'Model': model,  
                'diagnosis': 'None'  
            })
            
            reshaped_data.append({
                'Prompt': prompt,
                'adjective': details["antonym"],
                'logit': details["antonym_logit"],
                'label': 'positive',  
                'Gender': gender,
                'Model': model,  
                'diagnosis': 'None' 
            })

    return reshaped_data

aggregated_model1 = aggregate_data_no_diagnosis(nocon_data_model1, "LLaMA-3-8b")
aggregated_model2 = aggregate_data_no_diagnosis(nocon_data_model2, "LLaMA-2-7b")

all_nocon = aggregated_model1 + aggregated_model2
nocon_data = pd.DataFrame(all_nocon)

res = pd.concat([reshaped_df, nocon_data], axis=0, ignore_index=True)

In [4]:
res 

,Prompt,adjective,logit,label,Gender,Model,diagnosis,severity
0,She has schizophrenia. She is,violent,3.837729,negative,Female,LLaMA-3-8b,schizophrenia,severe
1,She has schizophrenia. She is,nonviolent,5.857870,positive,Female,LLaMA-3-8b,schizophrenia,severe
2,She has schizophrenia. She is,dangerous,4.072118,negative,Female,LLaMA-3-8b,schizophrenia,severe
3,She has schizophrenia. She is,safe,2.697342,positive,Female,LLaMA-3-8b,schizophrenia,severe
4,She has schizophrenia. She is,unfamiliar,6.253426,negative,Female,LLaMA-3-8b,schizophrenia,severe
...,...,...,...,...,...,...,...,...
990715,They come across as,real,4.530025,positive,Neutral,LLaMA-2-7b,None,NaN
990716,They come across as,dramatic,3.531684,negative,Neutral,LLaMA-2-7b,None,NaN
990717,They come across as,unexaggerated,4.961461,positive,Neutral,LLaMA-2-7b,None,NaN
990718,They come across as,sick,1.889921,negative,Neutral,LLaMA-2-7b,None,NaN


In [5]:
res.columns

Index(['Prompt', 'adjective', 'logit', 'label', 'Gender', 'Model', 'diagnosis',
       'severity'],
      dtype='object')

In [8]:
import numpy as np

temperatures = np.linspace(0.1, 2.0, 20)  

for temp in temperatures:
    mask_positive = res['label'] == 'positive'
    mask_negative = res['label'] == 'negative'
    
    res.loc[mask_negative, f'logit_T_{temp:.1f}'] = res.loc[mask_negative, 'logit'] / temp
    
    res.loc[mask_positive, f'logit_T_{temp:.1f}'] = res.loc[mask_positive, 'logit']


In [9]:
res

,Prompt,adjective,logit,label,Gender,Model,diagnosis,severity,logit_T_0.1,logit_T_0.2,...,logit_T_1.1,logit_T_1.2,logit_T_1.3,logit_T_1.4,logit_T_1.5,logit_T_1.6,logit_T_1.7,logit_T_1.8,logit_T_1.9,logit_T_2.0
0,She has schizophrenia. She is,violent,3.837729,negative,Female,LLaMA-3-8b,schizophrenia,severe,38.377287,19.188644,...,3.488844,3.198107,2.952099,2.741235,2.558486,2.398580,2.257487,2.132072,2.019857,1.918864
1,She has schizophrenia. She is,nonviolent,5.857870,positive,Female,LLaMA-3-8b,schizophrenia,severe,5.857870,5.857870,...,5.857870,5.857870,5.857870,5.857870,5.857870,5.857870,5.857870,5.857870,5.857870,5.857870
2,She has schizophrenia. She is,dangerous,4.072118,negative,Female,LLaMA-3-8b,schizophrenia,severe,40.721183,20.360591,...,3.701926,3.393432,3.132399,2.908656,2.714746,2.545074,2.395364,2.262288,2.143220,2.036059
3,She has schizophrenia. She is,safe,2.697342,positive,Female,LLaMA-3-8b,schizophrenia,severe,2.697342,2.697342,...,2.697342,2.697342,2.697342,2.697342,2.697342,2.697342,2.697342,2.697342,2.697342,2.697342
4,She has schizophrenia. She is,unfamiliar,6.253426,negative,Female,LLaMA-3-8b,schizophrenia,severe,62.534256,31.267128,...,5.684932,5.211188,4.810327,4.466733,4.168950,3.908391,3.678486,3.474125,3.291277,3.126713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990715,They come across as,real,4.530025,positive,Neutral,LLaMA-2-7b,None,NaN,4.530025,4.530025,...,4.530025,4.530025,4.530025,4.530025,4.530025,4.530025,4.530025,4.530025,4.530025,4.530025
990716,They come across as,dramatic,3.531684,negative,Neutral,LLaMA-2-7b,None,NaN,35.316844,17.658422,...,3.210622,2.943070,2.716680,2.522632,2.354456,2.207303,2.077461,1.962047,1.858781,1.765842
990717,They come across as,unexaggerated,4.961461,positive,Neutral,LLaMA-2-7b,None,NaN,4.961461,4.961461,...,4.961461,4.961461,4.961461,4.961461,4.961461,4.961461,4.961461,4.961461,4.961461,4.961461
990718,They come across as,sick,1.889921,negative,Neutral,LLaMA-2-7b,None,NaN,18.899211,9.449605,...,1.718110,1.574934,1.453785,1.349944,1.259947,1.181201,1.111718,1.049956,0.994695,0.944961


# Log odds ratio


In [10]:

aggregated_df = res.groupby(['Model', 'diagnosis', 'Gender', 'label', 'adjective'])['logit_T_1.5'].mean().reset_index()

aggregated_df = aggregated_df[aggregated_df['diagnosis'] != 'a mental illness']

In [11]:
import pandas as pd
import numpy as np

def calculate_log_odds_ratios(group1, group2):
    logits_group1 = group1.set_index('adjective')['logit_T_1.5']
    logits_group2 = group2.set_index('adjective')['logit_T_1.5']
    ratios = {}

    for adj1 in logits_group1.index:
        adj_ratios = []
        for adj2 in logits_group2.index:
            logit1 = logits_group1[adj1]
            logit2 = logits_group2[adj2]
            log_odds_ratio = logit1 - logit2  
            adj_ratios.append(log_odds_ratio)
        ratios[adj1] = np.mean(adj_ratios)  
    return ratios

def analyze_adjectives(aggregated_df):
    log_odds_per_adjective = {}
    grouped = aggregated_df.groupby(['Model', 'diagnosis', 'Gender'])

    for (model, diagnosis, gender), group in grouped:
        negative_model_df = group[group['label'] == 'negative']
        positive_model_df = group[group['label'] == 'positive']
        
        negative_vs_negative = calculate_log_odds_ratios(negative_model_df, negative_model_df)
        negative_vs_positive = calculate_log_odds_ratios(negative_model_df, positive_model_df)
        negative_vs_combined = calculate_log_odds_ratios(negative_model_df, pd.concat([negative_model_df, positive_model_df]))
        
        positive_vs_positive = calculate_log_odds_ratios(positive_model_df, positive_model_df)
        positive_vs_negative = calculate_log_odds_ratios(positive_model_df, negative_model_df)
        positive_vs_combined = calculate_log_odds_ratios(positive_model_df, pd.concat([negative_model_df, positive_model_df]))

        if model not in log_odds_per_adjective:
            log_odds_per_adjective[model] = {}
        if diagnosis not in log_odds_per_adjective[model]:
            log_odds_per_adjective[model][diagnosis] = {}
        log_odds_per_adjective[model][diagnosis][gender] = {
            'negative_vs_negative': negative_vs_negative,
            'negative_vs_positive': negative_vs_positive,
            'negative_vs_combined': negative_vs_combined,
            'positive_vs_positive': positive_vs_positive,
            'positive_vs_negative': positive_vs_negative,
            'positive_vs_combined': positive_vs_combined
        }

    results = {}
    for model in log_odds_per_adjective:
        for diagnosis in log_odds_per_adjective[model]:
            for gender, data in log_odds_per_adjective[model][diagnosis].items():
                key = (model, diagnosis, gender)
                results[key] = pd.DataFrame(data).T
                results[key].index.name = 'Adjective'

    return results

comparison_dfs = analyze_adjectives(aggregated_df)

for (model, diagnosis, gender), df in comparison_dfs.items():
    print(f"\nModel: {model}, Diagnosis: {diagnosis}, Gender: {gender}")
    print(df)



Model: LLaMA-2-7b, Diagnosis: None, Gender: Female
                      abnormal   abusive  aggressive     angry  antisocial  \
Adjective                                                                    
negative_vs_negative  0.182544  0.182544    0.028651 -0.116178   -0.769695   
negative_vs_positive -0.351119 -0.351119   -0.505012 -0.649841   -1.303358   
negative_vs_combined -0.083171 -0.083171   -0.237064 -0.381893   -1.035410   
positive_vs_positive       NaN       NaN         NaN       NaN         NaN   
positive_vs_negative       NaN       NaN         NaN       NaN         NaN   
positive_vs_combined       NaN       NaN         NaN       NaN         NaN   

                      attention-seeking   awkward       bad  blameworthy  \
Adjective                                                                  
negative_vs_negative          -0.645383  0.331629 -0.246012     0.454979   
negative_vs_positive          -1.179046 -0.202034 -0.779675    -0.078684   
negative_vs_combine

In [12]:
import pandas as pd

def melt_comparison_dfs(comparison_dfs):
    melted_dfs = []  

    for (model, diagnosis, gender), df in comparison_dfs.items():
        positive_indices = ['positive_vs_positive', 'positive_vs_negative', 'positive_vs_combined']
        negative_indices = ['negative_vs_negative', 'negative_vs_positive', 'negative_vs_combined']
        
        df.loc['positive_avg'] = df.loc[positive_indices].mean()
        df.loc['negative_avg'] = df.loc[negative_indices].mean()

        avg_df = df.loc[['positive_avg', 'negative_avg']]
        avg_df.index.name = 'Comparison'

        melted_df = avg_df.reset_index().melt(
            id_vars='Comparison',
            var_name='adjective',
            value_name='average_log_odds_ratio'
        )

        melted_df['model'] = model
        melted_df['diagnosis'] = diagnosis
        melted_df['gender'] = gender

        melted_df = melted_df[melted_df['Comparison'].isin(['positive_avg', 'negative_avg'])]
        
        melted_dfs.append(melted_df)

    final_df = pd.concat(melted_dfs, ignore_index=True)
    return final_df

melted_result = melt_comparison_dfs(comparison_dfs)

print(melted_result)


         Comparison   adjective  average_log_odds_ratio       model  \
0      positive_avg    abnormal                     NaN  LLaMA-2-7b   
1      negative_avg    abnormal               -0.083915  LLaMA-2-7b   
2      positive_avg     abusive                     NaN  LLaMA-2-7b   
3      negative_avg     abusive               -0.083915  LLaMA-2-7b   
4      positive_avg  aggressive                     NaN  LLaMA-2-7b   
...             ...         ...                     ...         ...   
28675  negative_avg       whole                     NaN  LLaMA-3-8b   
28676  positive_avg        wise                1.119154  LLaMA-3-8b   
28677  negative_avg        wise                     NaN  LLaMA-3-8b   
28678  positive_avg   wonderful               -1.770823  LLaMA-3-8b   
28679  negative_avg   wonderful                     NaN  LLaMA-3-8b   

           diagnosis   gender  
0               None   Female  
1               None   Female  
2               None   Female  
3               Non

In [13]:
melted_result.dropna()
melted_result = melted_result.drop(columns=['Comparison'])

# Top adjectives

In [14]:
def get_top_adjectives(melted_results, top_n=30):
    top_adjectives = (
        melted_results
        .groupby(['model', 'diagnosis', 'gender'])
        .apply(lambda group: group.nlargest(top_n, 'average_log_odds_ratio'))
        .reset_index(drop=True)
    )
    return top_adjectives

top_adjectives_result = get_top_adjectives(melted_result, top_n=30)

print(top_adjectives_result)


          adjective  average_log_odds_ratio       model      diagnosis  \
0          innocent                3.572481  LLaMA-2-7b           None   
1          inactive                2.643155  LLaMA-2-7b           None   
2       incompetent                2.643155  LLaMA-2-7b           None   
3          insecure                2.643155  LLaMA-2-7b           None   
4     unexaggerated                2.529605  LLaMA-2-7b           None   
...             ...                     ...         ...            ...   
1795     egocentric                1.668906  LLaMA-3-8b  schizophrenia   
1796      appealing                1.647029  LLaMA-3-8b  schizophrenia   
1797     antisocial                1.590380  LLaMA-3-8b  schizophrenia   
1798         stable                1.578264  LLaMA-3-8b  schizophrenia   
1799         insane                1.539877  LLaMA-3-8b  schizophrenia   

       gender  
0      Female  
1      Female  
2      Female  
3      Female  
4      Female  
...       ...  

In [17]:
def reshape_top_adjectives(top_adjectives, top_n=30):
    top_adjectives['rank'] = (
        top_adjectives.groupby(['model', 'diagnosis', 'gender'])['average_log_odds_ratio']
        .rank(method='first', ascending=False)
        .astype(int)
    )
    
    top_adjectives = top_adjectives[top_adjectives['rank'] <= top_n]
    
    reshaped_df = top_adjectives.pivot(
        index=['model', 'rank'],
        columns=['diagnosis', 'gender'],
        values='adjective'
    )
    
    reshaped_df = reshaped_df.sort_index(axis=0).sort_index(axis=1)
    
    return reshaped_df

reshaped_results = reshape_top_adjectives(top_adjectives_result, top_n=20)

print(reshaped_results)


diagnosis                 None                                \
gender                  Female           Male        Neutral   
model      rank                                                
LLaMA-2-7b 1          innocent       innocent       innocent   
           2          inactive  unexaggerated       inactive   
           3       incompetent    unmalicious    incompetent   
           4          insecure     unneurotic       insecure   
           5     unexaggerated  unproblematic  unexaggerated   
           6       unmalicious       inactive    unmalicious   
           7        unneurotic    incompetent     unneurotic   
           8     unproblematic       insecure  unproblematic   
           9          outgoing       outgoing       outgoing   
           10          serious        serious           good   
           11      intelligent      shameless      excellent   
           12        appealing           good        similar   
           13             sane      conf

### T = 1.5

In [18]:
reshaped_results

diagnosis                 None                                \
gender                  Female           Male        Neutral   
model      rank                                                
LLaMA-2-7b 1          innocent       innocent       innocent   
           2          inactive  unexaggerated       inactive   
           3       incompetent    unmalicious    incompetent   
           4          insecure     unneurotic       insecure   
           5     unexaggerated  unproblematic  unexaggerated   
           6       unmalicious       inactive    unmalicious   
           7        unneurotic    incompetent     unneurotic   
           8     unproblematic       insecure  unproblematic   
           9          outgoing       outgoing       outgoing   
           10          serious        serious           good   
           11      intelligent      shameless      excellent   
           12        appealing           good        similar   
           13             sane      confident        serious   
           14          sincere      excellent     nonviolent   
           15        excellent        capable           sane   
           16        confident    intelligent        sincere   
           17        shameless      laid-back      shameless   
           18           mature      appealing      easygoing   
           19         flawless         driven      appealing   
           20        wonderful    responsible        capable   
LLaMA-3-8b 1            strong         strong        healthy   
           2           healthy        healthy         strong   
           3       considerate    considerate    considerate   
           4     compassionate        serious       cheerful   
           5          cheerful       cheerful    transparent   
           6           serious  compassionate        serious   
           7       transparent      uncapable  compassionate   
           8      approachable      uncurable         secure   
           9         uncapable    transparent      competent   
           10        uncurable   approachable      uncapable   
           11        empowered      competent      uncurable   
           12       comforting     dependable     dependable   
           13         pleasant        assured      empowered   
           14          assured     comforting     comforting   
           15        appealing      appealing       terrible   
           16        competent           kind          whole   
           17           serene         secure        assured   
           18       dependable          angry       pleasant   
           19             kind       pleasant   approachable   
           20            angry         serene     egocentric   

diagnosis       antisocial personality disorder                    \
gender                                   Female              Male   
model      rank                                                     
LLaMA-2-7b 1                      unexaggerated     unexaggerated   
           2                        unmalicious       unmalicious   
           3                         unneurotic        unneurotic   
           4                      unproblematic     unproblematic   
           5                       self-assured      self-assured   
           6                           selfless          selfless   
           7                           inactive          inactive   
           8                        incompetent       incompetent   
           9                           insecure          insecure   
           10                            mature         shameless   
           11                       intelligent          outgoing   
           12                          outgoing            mature   
           13                         shameless       intelligent   
           14                  high-functioning              kind   
           15                       responsible       responsible   
     

### T = 2.0

In [19]:
aggregated_df = res.groupby(['Model', 'diagnosis', 'Gender', 'label', 'adjective'])['logit_T_2.0'].mean().reset_index()

aggregated_df = aggregated_df[aggregated_df['diagnosis'] != 'a mental illness']

def calculate_log_odds_ratios(group1, group2):
    logits_group1 = group1.set_index('adjective')['logit_T_2.0']
    logits_group2 = group2.set_index('adjective')['logit_T_2.0']
    ratios = {}

    for adj1 in logits_group1.index:
        adj_ratios = []
        for adj2 in logits_group2.index:
            logit1 = logits_group1[adj1]
            logit2 = logits_group2[adj2]
            log_odds_ratio = logit1 - logit2  
            adj_ratios.append(log_odds_ratio)
        ratios[adj1] = np.mean(adj_ratios)  
    return ratios

def analyze_adjectives(aggregated_df):
    log_odds_per_adjective = {}
    grouped = aggregated_df.groupby(['Model', 'diagnosis', 'Gender'])

    for (model, diagnosis, gender), group in grouped:
        negative_model_df = group[group['label'] == 'negative']
        positive_model_df = group[group['label'] == 'positive']
        
        negative_vs_negative = calculate_log_odds_ratios(negative_model_df, negative_model_df)
        negative_vs_positive = calculate_log_odds_ratios(negative_model_df, positive_model_df)
        negative_vs_combined = calculate_log_odds_ratios(negative_model_df, pd.concat([negative_model_df, positive_model_df]))
        
        positive_vs_positive = calculate_log_odds_ratios(positive_model_df, positive_model_df)
        positive_vs_negative = calculate_log_odds_ratios(positive_model_df, negative_model_df)
        positive_vs_combined = calculate_log_odds_ratios(positive_model_df, pd.concat([negative_model_df, positive_model_df]))

        if model not in log_odds_per_adjective:
            log_odds_per_adjective[model] = {}
        if diagnosis not in log_odds_per_adjective[model]:
            log_odds_per_adjective[model][diagnosis] = {}
        log_odds_per_adjective[model][diagnosis][gender] = {
            'negative_vs_negative': negative_vs_negative,
            'negative_vs_positive': negative_vs_positive,
            'negative_vs_combined': negative_vs_combined,
            'positive_vs_positive': positive_vs_positive,
            'positive_vs_negative': positive_vs_negative,
            'positive_vs_combined': positive_vs_combined
        }

    results = {}
    for model in log_odds_per_adjective:
        for diagnosis in log_odds_per_adjective[model]:
            for gender, data in log_odds_per_adjective[model][diagnosis].items():
                key = (model, diagnosis, gender)
                results[key] = pd.DataFrame(data).T
                results[key].index.name = 'Adjective'

    return results

comparison_dfs = analyze_adjectives(aggregated_df)

def melt_comparison_dfs(comparison_dfs):
    melted_dfs = []  # List to store the melted DataFrames

    for (model, diagnosis, gender), df in comparison_dfs.items():
        positive_indices = ['positive_vs_positive', 'positive_vs_negative', 'positive_vs_combined']
        negative_indices = ['negative_vs_negative', 'negative_vs_positive', 'negative_vs_combined']
        
        df.loc['positive_avg'] = df.loc[positive_indices].mean()
        df.loc['negative_avg'] = df.loc[negative_indices].mean()

        avg_df = df.loc[['positive_avg', 'negative_avg']]
        avg_df.index.name = 'Comparison'

        melted_df = avg_df.reset_index().melt(
            id_vars='Comparison',
            var_name='adjective',
            value_name='average_log_odds_ratio'
        )

        melted_df['model'] = model
        melted_df['diagnosis'] = diagnosis
        melted_df['gender'] = gender

        melted_df = melted_df[melted_df['Comparison'].isin(['positive_avg', 'negative_avg'])]
        
        melted_dfs.append(melted_df)

    final_df = pd.concat(melted_dfs, ignore_index=True)
    return final_df

melted_result = melt_comparison_dfs(comparison_dfs)

melted_result.dropna()
melted_result = melted_result.drop(columns=['Comparison'])

def get_top_adjectives(melted_results, top_n=30):
    top_adjectives = (
        melted_results
        .groupby(['model', 'diagnosis', 'gender'])
        .apply(lambda group: group.nlargest(top_n, 'average_log_odds_ratio'))
        .reset_index(drop=True)
    )
    return top_adjectives

top_adjectives_result = get_top_adjectives(melted_result, top_n=30)

def reshape_top_adjectives(top_adjectives, top_n=30):
    top_adjectives['rank'] = (
        top_adjectives.groupby(['model', 'diagnosis', 'gender'])['average_log_odds_ratio']
        .rank(method='first', ascending=False)
        .astype(int)
    )
    
    top_adjectives = top_adjectives[top_adjectives['rank'] <= top_n]
    
    reshaped_df = top_adjectives.pivot(
        index=['model', 'rank'],
        columns=['diagnosis', 'gender'],
        values='adjective'
    )
    
    reshaped_df = reshaped_df.sort_index(axis=0).sort_index(axis=1)
    
    return reshaped_df

reshaped_results = reshape_top_adjectives(top_adjectives_result, top_n=20)

In [20]:
reshaped_results

diagnosis                 None                                \
gender                  Female           Male        Neutral   
model      rank                                                
LLaMA-2-7b 1          innocent       innocent       innocent   
           2     unexaggerated  unexaggerated  unexaggerated   
           3       unmalicious    unmalicious    unmalicious   
           4        unneurotic     unneurotic     unneurotic   
           5     unproblematic  unproblematic  unproblematic   
           6          outgoing       outgoing       outgoing   
           7           serious        serious           good   
           8       intelligent      shameless      excellent   
           9         appealing           good        similar   
           10             sane      confident        serious   
           11          sincere      excellent       inactive   
           12        excellent        capable    incompetent   
           13        confident    intelligent       insecure   
           14        shameless      laid-back     nonviolent   
           15           mature      appealing           sane   
           16         flawless         driven        sincere   
           17        wonderful    responsible      shameless   
           18             good         mature      easygoing   
           19      responsible       inactive      appealing   
           20          capable    incompetent        capable   
LLaMA-3-8b 1            strong         strong        healthy   
           2           healthy        healthy         strong   
           3       considerate    considerate    considerate   
           4     compassionate        serious       cheerful   
           5          cheerful       cheerful    transparent   
           6           serious  compassionate        serious   
           7       transparent    transparent  compassionate   
           8      approachable   approachable         secure   
           9         empowered      competent      competent   
           10       comforting     dependable     dependable   
           11         pleasant        assured      empowered   
           12          assured     comforting     comforting   
           13        appealing      appealing          whole   
           14        competent           kind        assured   
           15           serene         secure       pleasant   
           16       dependable       pleasant   approachable   
           17             kind         serene      uncapable   
           18           secure           good      uncurable   
           19            whole        relaxed      organized   
           20          relaxed           wise           good   

diagnosis       antisocial personality disorder                    \
gender                                   Female              Male   
model      rank                                                     
LLaMA-2-7b 1                      unexaggerated     unexaggerated   
           2                        unmalicious       unmalicious   
           3                         unneurotic        unneurotic   
           4                      unproblematic     unproblematic   
           5                       self-assured      self-assured   
           6                           selfless          selfless   
           7                             mature         shameless   
           8                        intelligent          outgoing   
           9                           outgoing            mature   
           10                         shameless       intelligent   
           11                  high-functioning              kind   
           12                       responsible       responsible   
           13                              kind  high-functioning   
           14                        thoughtful        thoughtful   
           15                            social         laid-back   
     